<a href="https://colab.research.google.com/github/matthewtech-o/HAMOYE-INTERNSHIP/blob/main/Hamoye_Stage_B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import the libraries for data preprocessing and dealing with numerical values
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv('energydata_complete.csv')
data.head()

,date,Appliances,lights,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,RH_5,T6,RH_6,T7,RH_7,T8,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,rv1,rv2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [3]:
# to rename the column headings present in our dataset, we need to create a dictionary containing the values to replace the headings with
column_names = {'date':'date', 'Appliances': 'Appliances', 
                'lights':  'lights', 'T1': 'Kitchen_Area_Temp', 'RH_1': 'Kitchen_Area_Humidity',
                'T2': 'Living_Room_Temp', 'RH_2': 'Living_Room_Humidity', 
                'T3': 'Laundry_Room_Temp', 'RH_3': 'Laundry_Room_Humidity','T4':'Office_Room_Temp', 'RH_4':'Office_Room_Humidity',  
                'T5': 'Bathroom_Temp', 'RH_5': 'Bathroom_Humidity', 'T6':'North_Side_Temp','RH_6': 'North_Side_Humidity', 
               'T7':'Ironing_Room_Temp','RH_7': 'Ironing_Room_Humidity','T8':'Teenager_Room_Temp', 'RH_8': 'Teenager_Room_Humidity',
               'T9': 'Parents_Room_Temp', 'RH_9': 'Parents_Room_Humidity', 'T_out': 'Outside_Temperature',
                'Press_mm_hg': 'Weather_Station_Pressure', 'Windspeed': 'Windspeed','Visibility': 'Visibility', 
                'RH_out': 'Outside_Humidity','Tdewpoint': 'Dewpoint', 'rv1': 'Random_variable_1', 'rv2': 'Random_variable_2'}
data = data.rename(columns=column_names)
data.head()

,date,Appliances,lights,Kitchen_Area_Temp,Kitchen_Area_Humidity,Living_Room_Temp,Living_Room_Humidity,Laundry_Room_Temp,Laundry_Room_Humidity,Office_Room_Temp,Office_Room_Humidity,Bathroom_Temp,Bathroom_Humidity,North_Side_Temp,North_Side_Humidity,Ironing_Room_Temp,Ironing_Room_Humidity,Teenager_Room_Temp,Teenager_Room_Humidity,Parents_Room_Temp,Parents_Room_Humidity,Outside_Temperature,Weather_Station_Pressure,Outside_Humidity,Windspeed,Visibility,Dewpoint,Random_variable_1,Random_variable_2
0,2016-01-11 17:00:00,60,30,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,2016-01-11 17:10:00,60,30,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,2016-01-11 17:20:00,50,30,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,2016-01-11 17:30:00,50,40,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,2016-01-11 17:40:00,60,40,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [4]:
data=data.drop(['date','lights'],axis=1)

In [5]:
data.head()

,Appliances,Kitchen_Area_Temp,Kitchen_Area_Humidity,Living_Room_Temp,Living_Room_Humidity,Laundry_Room_Temp,Laundry_Room_Humidity,Office_Room_Temp,Office_Room_Humidity,Bathroom_Temp,Bathroom_Humidity,North_Side_Temp,North_Side_Humidity,Ironing_Room_Temp,Ironing_Room_Humidity,Teenager_Room_Temp,Teenager_Room_Humidity,Parents_Room_Temp,Parents_Room_Humidity,Outside_Temperature,Weather_Station_Pressure,Outside_Humidity,Windspeed,Visibility,Dewpoint,Random_variable_1,Random_variable_2
0,60,19.89,47.596667,19.2,44.790000,19.79,44.730000,19.000000,45.566667,17.166667,55.20,7.026667,84.256667,17.200000,41.626667,18.2,48.900000,17.033333,45.53,6.600000,733.5,92.0,7.000000,63.000000,5.3,13.275433,13.275433
1,60,19.89,46.693333,19.2,44.722500,19.79,44.790000,19.000000,45.992500,17.166667,55.20,6.833333,84.063333,17.200000,41.560000,18.2,48.863333,17.066667,45.56,6.483333,733.6,92.0,6.666667,59.166667,5.2,18.606195,18.606195
2,50,19.89,46.300000,19.2,44.626667,19.79,44.933333,18.926667,45.890000,17.166667,55.09,6.560000,83.156667,17.200000,41.433333,18.2,48.730000,17.000000,45.50,6.366667,733.7,92.0,6.333333,55.333333,5.1,28.642668,28.642668
3,50,19.89,46.066667,19.2,44.590000,19.79,45.000000,18.890000,45.723333,17.166667,55.09,6.433333,83.423333,17.133333,41.290000,18.1,48.590000,17.000000,45.40,6.250000,733.8,92.0,6.000000,51.500000,5.0,45.410389,45.410389
4,60,19.89,46.333333,19.2,44.530000,19.79,45.000000,18.890000,45.530000,17.200000,55.09,6.366667,84.893333,17.200000,41.230000,18.1,48.590000,17.000000,45.40,6.133333,733.9,92.0,5.666667,47.666667,4.9,10.084097,10.084097


In [6]:
#Scaling the dataset, we use the MinMax scaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data)

#to convert scaled dataset into a pandas dataframe
data = pd.DataFrame(scaled_data,columns=data.columns)

X=data.drop('Appliances',axis=1)
y=data['Appliances']


#to divide datset into train and testing data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)

#import the linear model
from sklearn.linear_model import LinearRegression
linear_model = LinearRegression()
linear_model.fit(X_train,y_train)

#to predict the test set
predicted_values = linear_model.predict(X_test)

In [7]:
#Getting the Mean Absolute Error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test,predicted_values)
round(mae,3)

0.05

In [8]:
#Getting the Residual sum of squares
rss=np.sum(np.square(y_test-predicted_values))
round(rss,2)

45.35

In [9]:
#Getting the root mean square error
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test,predicted_values))
round(rmse,3)

0.088

In [10]:
#Getting the Coefficient of determination
from sklearn.metrics import r2_score
r2_score = r2_score(y_test,predicted_values)
round(r2_score,2)

0.15

In [12]:
def get_weights_df(model, fit, col_name):
    
    #this function returns the weight of every feature
    weights = pd.Series(model.coef_, fit.columns).sort_values()
    weights_df = pd.DataFrame(weights).reset_index()
    weights_df.columns = ['Features', col_name]
    weights_df[col_name].round(3)
    return weights_df

In [13]:
get_weights_df(linear_model,X,'Linear_weight_coefficients')

,Features,Linear_weight_coefficients
0,Living_Room_Humidity,-0.456698
1,Outside_Temperature,-0.321860
2,Living_Room_Temp,-0.236178
3,Parents_Room_Temp,-0.189941
4,Teenager_Room_Humidity,-0.157595
5,Outside_Humidity,-0.077671
6,Ironing_Room_Humidity,-0.044614
7,Parents_Room_Humidity,-0.039800
8,Bathroom_Temp,-0.015657
9,Kitchen_Area_Temp,-0.003281


In [14]:
#Training a Ridge regreession model with alpha=0.4
from sklearn.linear_model import Ridge
ridge_model = Ridge(alpha=0.4)
ridge_model.fit(X_train,y_train)

Ridge(alpha=0.4, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001)

In [15]:
ridge_predict = ridge_model.predict(X_test)

In [16]:
rmse=np.sqrt(mean_squared_error(y_test,ridge_predict))
round(rmse,3)

0.088

In [17]:
#Training a Lasso regreession model with alpha=0.001
from sklearn.linear_model import Lasso
lasso_model = Lasso(alpha=0.001)
lasso_model.fit(X_train,y_train)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [18]:
len(lasso_model.coef_[lasso_model.coef_ !=0])

4

In [19]:
lasso_predict = lasso_model.predict(X_test)
lasso_predict

array([0.07370267, 0.08143458, 0.07716072, ..., 0.07792848, 0.09034412,
       0.08359255])

In [20]:
#Root mean square for Lasso prediction
rmse=np.sqrt(mean_squared_error(y_test,lasso_predict))
round(rmse,3)

0.094